In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration,Action, row_col
#from kaggle_environments.envs.hungry_geese.hungry_geese import adjacent_positions,min_distance

In [ ]:
%%writefile submission.py
import random
position_decode={}
for x in range(77):
    position_decode[x]=[x//11,x%11]
last_turn=['NORTH','NORTH','NORTH','NORTH']
def pairwise_dist(c1,c2):  #calculates between c1 and c2
    return abs(c1[0]-c2[0])+abs(c1[1]-c2[1])    
def agent(obs_dict, config_dict):
        g=obs_dict['geese']
        f=obs_dict['food']
        index=obs_dict['index']
        my_goose_pos=position_decode[g[index][0]]
        print(index)
        print("Position",g[index])
        g.pop(index)
        geese=[]
        for x in g:
            for y in x:
                geese.append(position_decode[y])
        food=[]
        for x in f:
            food.append(position_decode[x])
        my_goose_food_dist=[]
        eatable=[]
        for x in food:
            my_goose_food_dist.append(pairwise_dist(x,my_goose_pos))   
        for x in range(len(food)):
            dist_from_food=[]
            for y in geese:
                dist_from_food.append(pairwise_dist(food[x],y))
            
            eatable.append(my_goose_food_dist[x]<min(dist_from_food))
            
        try:
            food_available=eatable[0]
            goto_dist=my_goose_food_dist[0]
            goto=0
            for x in range(1,len(eatable)):
                if eatable[x] and my_goose_food_dist[x]<goto_dist:
                    food_available=eatable[x]
                    goto_dist=my_goose_food_dist[x]
                    goto=x
        except IndexError:
            food_available=False
        print(my_goose_pos)
        print("Food Available ",food_available)
        possible_moves=['NORTH','SOUTH','EAST','WEST']
        l=last_turn[index]
        if l=='SOUTH':
            possible_moves.remove("NORTH")
        elif l=='NORTH':
            possible_moves.remove("SOUTH")
        elif l=='WEST':
            possible_moves.remove("EAST")
        elif l=='EAST':
            possible_moves.remove("WEST")
        if food_available:
            go_to_food=food[goto]
            print('Food Position ',go_to_food)
            print("Last move ",l)
            if go_to_food[0]>my_goose_pos[0] and "SOUTH" in possible_moves:
                last_turn[index]='SOUTH'
                return 'SOUTH'
            elif go_to_food[0]<my_goose_pos[0] and "NORTH" in possible_moves:
                last_turn[index]='NORTH'
                return 'NORTH'
            elif go_to_food[1]<my_goose_pos[1] and "WEST" in possible_moves:
                last_turn[index]='WEST'
                return 'WEST'
            elif go_to_food[1]>my_goose_pos[1] and "EAST" in possible_moves:
                last_turn[index]="EAST"
                return 'EAST'
        else:
            dist_from_enemy=[]
            for x in geese:
                dist_from_enemy.append(pairwise_dist(x,my_goose_pos))
            closest_goose=min(dist_from_enemy)
            
            pos_closest=geese[dist_from_enemy.index(closest_goose)]
            if pos_closest[0]>my_goose_pos[0] and 'SOUTH' in possible_moves:
                possible_moves.remove("SOUTH")
            if pos_closest[0]<my_goose_pos[0] and 'NORTH' in possible_moves and len(possible_moves)>1:
                possible_moves.remove("NORTH")
            if pos_closest[1]>my_goose_pos[1] and 'WEST' in possible_moves and len(possible_moves)>1:
                possible_moves.remove("WEST")
            if pos_closest[1]>my_goose_pos[1] and 'EAST' in possible_moves and len(possible_moves)>1:
                possible_moves.remove("EAST")
            print("Last turn",last_turn[index])
            print('Possible moves',possible_moves)
            print("-----------------------------------")
            l=random.choice(possible_moves)
            last_turn[index]=l
            return l
        

In [ ]:
import kaggle_environments
from kaggle_environments import make, evaluate, utils

env = make("hungry_geese", debug=True) #set debug to True to see agent internals each step

env.reset()
env.run(["./submission.py","./submission.py","./submission.py",'./submission.py'])
env.render(mode="ipython",width=700, height=600)